In [208]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [209]:
# Cargar el archivo train.csv desde la misma carpeta
td = pd.read_csv('dataset_concatenado_con_desviacion.csv', low_memory=False)

# Ver las primeras filas del archivo
td.head()

,id,destination,route,alternate_route,departure_time,arrival_time,current_time,traffic_streets,distance_to_next_stop,time_estimate,...,incident_description,deviation,fecha,medio_transporte,linea,pasajeros,tiempo_espera_min,puntualidad,incidencias,desviacion
0,1.0,Puerta del Sol,Linea 3,Linea 9,19:49,20:00,19:58,Paseo del Prado,2.12,55.0,...,Tráfico denso,sí,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.789186
1,2.0,Plaza Castilla,Linea 9,Linea 7,02:23,02:36,02:50,Calle Atocha,2.44,21.0,...,Retraso por obras,sí,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.604428
2,3.0,Puerta del Sol,Linea 4,Linea 2,18:58,19:31,19:11,Calle de Bravo Murillo,0.60,29.0,...,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.053541
3,4.0,Retiro,Linea 1,Linea 8,13:40,14:00,13:43,Calle de Guzmán el Bueno,4.56,29.0,...,NaN,sí,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.769928
4,5.0,Retiro,Linea 2,Linea 5,05:58,06:19,06:13,Calle Alcalá,3.57,48.0,...,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.175909


In [210]:
td.shape

(200000, 30)

In [211]:
td.columns

Index(['id', 'destination', 'route', 'alternate_route', 'departure_time',
       'arrival_time', 'current_time', 'traffic_streets',
       'distance_to_next_stop', 'time_estimate', 'day_of_week', 'time_of_day',
       'traffic_status', 'traffic_delay', 'vehicle_type',
       'alternate_route_available', 'alternate_route_time_estimate',
       'route_efficiency', 'incident_occurred', 'incident_type',
       'incident_description', 'deviation', 'fecha', 'medio_transporte',
       'linea', 'pasajeros', 'tiempo_espera_min', 'puntualidad', 'incidencias',
       'desviacion'],
      dtype='object')

In [212]:
td.drop(['id', 'route',  'deviation', 'pasajeros','alternate_route','distance_to_next_stop','current_time','day_of_week','time_of_day',
       'route_efficiency','medio_transporte','pasajeros','desviacion','arrival_time'],axis = 1, inplace = True)
td.head()

,destination,departure_time,traffic_streets,time_estimate,traffic_status,traffic_delay,vehicle_type,alternate_route_available,alternate_route_time_estimate,incident_occurred,incident_type,incident_description,fecha,linea,tiempo_espera_min,puntualidad,incidencias
0,Puerta del Sol,19:49,Paseo del Prado,55.0,congestionado,12.0,metro,sí,63.0,sí,accidente,Tráfico denso,NaN,NaN,NaN,NaN,NaN
1,Plaza Castilla,02:23,Calle Atocha,21.0,moderado,0.0,autobús,no,NaN,sí,avería,Retraso por obras,NaN,NaN,NaN,NaN,NaN
2,Puerta del Sol,18:58,Calle de Bravo Murillo,29.0,fluido,0.0,autobús,sí,43.0,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Retiro,13:40,Calle de Guzmán el Bueno,29.0,congestionado,14.0,autobús,no,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Retiro,05:58,Calle Alcalá,48.0,congestionado,0.0,autobús,no,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN


***RELLENAR VALORES NULOS***

In [213]:
td.isnull().sum()

destination                      100000
departure_time                   100000
traffic_streets                  100000
time_estimate                    100000
traffic_status                   100000
traffic_delay                    100000
vehicle_type                     100000
alternate_route_available        100000
alternate_route_time_estimate    150114
incident_occurred                100000
incident_type                    149914
incident_description             149914
fecha                            100000
linea                            100000
tiempo_espera_min                100000
puntualidad                      100000
incidencias                      100000
dtype: int64

In [214]:
td.dtypes

destination                       object
departure_time                    object
traffic_streets                   object
time_estimate                    float64
traffic_status                    object
traffic_delay                    float64
vehicle_type                      object
alternate_route_available         object
alternate_route_time_estimate    float64
incident_occurred                 object
incident_type                     object
incident_description              object
fecha                             object
linea                             object
tiempo_espera_min                float64
puntualidad                       object
incidencias                       object
dtype: object

In [215]:

# Para columnas de fecha
for col in ['current_time', 'fecha']:
    if col not in td.columns:
        td[col] = pd.Timestamp('2023-10-01')
    else:
        td[col] = td[col].fillna(pd.Timestamp('2023-10-01'))
td['traffic_delay'] = td['traffic_delay'].fillna(td['traffic_delay'].mean())


columnas_a_llenar = {
    'day_of_week': 'Lunes',
    'time_of_day': '08:00',
    'traffic_status': 'Desconocido',
    'incident_occurred': 0,
    'incident_type': 'Ninguno',
    'incident_description': 'Sin descripción'
}


# Rellenar columnas de fechas con un valor constante o el valor más frecuente
td['current_time'] = td['current_time'].fillna(pd.Timestamp('2023-10-01'))  # Cambia la fecha según sea necesario
td['fecha'] = td['fecha'].fillna(pd.Timestamp('2023-10-01'))


# Rellenar el estado del tráfico
td['traffic_status'] = td['traffic_status'].fillna('Desconocido')  # O la moda

# Rellenar incidencias
td['incident_occurred'] = td['incident_occurred'].fillna(0)  # Supón que 0 significa que no ocurrió
td['incident_type'] = td['incident_type'].fillna('Ninguno')  # O usa la moda
td['incident_description'] = td['incident_description'].fillna('Sin descripción')  # O la moda


In [216]:
def fill_dataframe_safely(td, columns_to_fill):
    for col, fill_strategy in columns_to_fill.items():
        if col not in td.columns:
            if isinstance(fill_strategy, (int, float)):
                td[col] = fill_strategy
            elif fill_strategy == 'mean':
                td[col] = np.nan  # Crear la columna con NaN para luego llenarla con la media
            else:
                td[col] = fill_strategy
        
        if fill_strategy == 'mean':
            td[col] = td[col].fillna(td[col].mean())
        else:
            td[col] = td[col].fillna(fill_strategy)
    
    return td

# Uso:
columns_to_fill = {
    'route_efficiency': 'mean',
    'deviation': 0,
    # Añade aquí otras columnas que quieras manejar
}

td = fill_dataframe_safely(td, columns_to_fill)

In [218]:

# Rellenar `traffic_streets` con un valor constante
td['traffic_streets'] = td['traffic_streets'].fillna('Sin información')

# Rellenar `vehicle_type` usando `apply()` para evitar errores
td['vehicle_type'] = td['vehicle_type'].apply(lambda x: x if pd.notna(x) else np.random.choice(vehicle_types))

# Rellenar `alternate_route_available` con "No"
td['alternate_route_available'] = td['alternate_route_available'].fillna('No')

# Rellenar `alternate_route_time_estimate` con la media de la columna
td['alternate_route_time_estimate'] = td['alternate_route_time_estimate'].fillna(td['alternate_route_time_estimate'].mean())

# Rellenar `puntualidad` con "0"
td['puntualidad'] = td['puntualidad'].fillna(0)

In [219]:
td.columns

Index(['destination', 'departure_time', 'traffic_streets', 'time_estimate',
       'traffic_status', 'traffic_delay', 'vehicle_type',
       'alternate_route_available', 'alternate_route_time_estimate',
       'incident_occurred', 'incident_type', 'incident_description', 'fecha',
       'linea', 'tiempo_espera_min', 'puntualidad', 'incidencias',
       'current_time', 'route_efficiency', 'deviation',
       'departure_time_minutes'],
      dtype='object')

In [220]:
td.isnull().sum()

destination                      100000
departure_time                   100000
traffic_streets                       0
time_estimate                    100000
traffic_status                        0
traffic_delay                         0
vehicle_type                          0
alternate_route_available             0
alternate_route_time_estimate         0
incident_occurred                     0
incident_type                         0
incident_description                  0
fecha                                 0
linea                            100000
tiempo_espera_min                100000
puntualidad                           0
incidencias                      100000
current_time                          0
route_efficiency                 200000
deviation                             0
departure_time_minutes           100000
dtype: int64

In [227]:
# Eliminar valores NaN y duplicados usando set
destination_list = list(set(td['destination'].dropna()))

print(destination_list)


['Plaza Castilla', 'Gran Via', 'El Prado', 'Retiro', 'Puerta del Sol', 'Plaza Mayor', 'Madrid Rio', 'Chamartin', 'Santiago Bernabeu', 'Atocha']


In [228]:
#rellenar valores nulos en una columna e funcion de otra
td['incidencias'] = td['incidencias'].fillna(td['deviation'])


In [229]:
# Lista de posibles destinos y rutas
destinations = ['Plaza Castilla', 'Gran Via', 'El Prado', 'Retiro', 'Puerta del Sol', 'Plaza Mayor', 'Madrid Rio', 'Chamartin', 'Santiago Bernabeu', 'Atocha']
linea = ['Linea 1', 'Linea 8', 'Linea 6', 'Linea 7', 'Linea 5', 'Linea 10', 'Linea 9', 'Linea 2', 'Linea 4', 'Linea 3']
vehicle_types = ['Bicicleta', 'Metro', 'Tren', 'Autobús']

In [230]:
td.isnull().sum()

destination                           0
departure_time                   100000
traffic_streets                       0
time_estimate                    100000
traffic_status                        0
traffic_delay                         0
vehicle_type                          0
alternate_route_available             0
alternate_route_time_estimate         0
incident_occurred                     0
incident_type                         0
incident_description                  0
fecha                                 0
linea                            100000
tiempo_espera_min                100000
puntualidad                           0
incidencias                           0
current_time                          0
route_efficiency                 200000
deviation                             0
departure_time_minutes           100000
dtype: int64

In [ ]:
from datetime import datetime, timedelta

def impute_dataframe(td, imputation_strategies):
    for column, strategy in imputation_strategies.items():
        if column not in td.columns:
            print(f"Advertencia: La columna '{column}' no existe en el DataFrame.")
            continue
        
        if callable(strategy):
            td[column] = td[column].fillna(strategy(td))
        elif isinstance(strategy, (list, np.ndarray)):
            td[column] = td[column].fillna(pd.Series(np.random.choice(strategy, size=len(td))))
        elif strategy == 'mean':
            td[column] = td[column].fillna(td[column].mean())
        elif strategy == 'median':
            td[column] = td[column].fillna(td[column].median())
        elif strategy == 'mode':
            td[column] = td[column].fillna(td[column].mode()[0])
        else:
            td[column] = td[column].fillna(strategy)
    
    return td

imputation_strategies = {
    'departure_time': lambda td: pd.to_datetime(datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(minutes=np.random.randint(0, 1440))),
    'time_estimate': 'mean',
    'linea': linea,
    'tiempo_espera_min': lambda td: np.random.randint(0, 15),
    'route_efficiency': 'median',
    'departure_time_minutes': lambda td: np.random.randint(0, 1440)
}

# Aplicar la imputación
td = impute_dataframe(td, imputation_strategies)